In [1]:
import numpy as np
import astropy
import gammapy
from gammapy.time import LightCurve, LightCurveEstimator
from gammapy.spectrum import (
    models, SpectrumExtraction, SpectrumObservationList, SpectrumFit, SpectrumResult,
    SpectrumEnergyGroupMaker, FluxPointEstimator,
)
from gammapy.utils.energy import EnergyBounds
import astropy.units as u
from gammapy.image import SkyImage, IACTBasicImageEstimator
from gammapy.background import RingBackgroundEstimator, ReflectedRegionsBackgroundEstimator
from regions import CircleSkyRegion
from astropy.coordinates import SkyCoord

In [2]:
import gammapy
from gammapy.data import DataStore
%matplotlib inline
import matplotlib.pyplot as plt

# Loading 1DC GPS data

In [3]:
import os
os.environ['CTADATA'] = '/photon1/users/labsaha/gammapy_analysis/data/1dc/'
data_store = DataStore.from_dir('$CTADATA/index/gps')
table = data_store.obs_table

# Selecting Data 

In [4]:
table = data_store.obs_table
pos_obs = SkyCoord(table['GLON_PNT'], table['GLAT_PNT'], frame='galactic', unit='deg')

Radius = 3.5 
lon_source = 135.6753 
lat_source = +01.0861 

pos_target = SkyCoord(lon_source, lat_source, frame='galactic', unit='deg')
offset = pos_target.separation(pos_obs).deg
#mask = (1 < offset) & (offset < 2)
mask = offset < Radius
table = table[mask]


obs_ids = table['OBS_ID']
obs_list = data_store.obs_list(obs_ids)


In [5]:
__To define on regions and off regions__

SyntaxError: invalid syntax (<ipython-input-5-70abd24f8521>, line 1)

In [ ]:
on_radius = 0.6 * u.deg
on_region = CircleSkyRegion(center=pos_target, radius=on_radius)

off_regions=[]

circle = CircleSkyRegion(center=SkyCoord(135.6753,01.0861,unit='deg',frame='fk5'),radius=0.15 *u.deg)

off_regions.append(circle)
    
# Exclusion regions
# Define reference image centered on the target

xref = pos_target.galactic.l.value
yref = pos_target.galactic.b.value
size = 7 * u.deg
binsz = 0.025 # degree per pixel
npix = int((size / binsz).value)

ref_image = SkyImage.empty(
         nxpix=npix, nypix=npix, binsz=binsz,
         xref=xref, yref=yref,
         proj='TAN', coordsys='GAL',
         )
print(ref_image)
exclusion_mask = ref_image.region_mask(on_region)

print(offset)
exclusion_mask.data = 1 - exclusion_mask.data
for off in off_regions:
        exclusion_mask.data -= ref_image.region_mask(off)

# To plot the exclusion mask  
#exclusion_mask.plot(add_cbar=True)

model = models.PowerLaw(
                    index=2. * u.Unit(''),
                    amplitude=1e-12 * u.Unit('cm-2 s-1 TeV-1'),
                    reference=1. * u.Unit('TeV')

                    )


# To calculate fluxes at different phases using a PhaseCurve class function

In [ ]:
# Calling PhaseCurve class 
from test_models_foldedLC import PhaseCurve

#period of the binary
period = 26.496

# Reference MJD value where orbital phase of the binary is considered zero 
T0 = 43366.275

a = PhaseCurve(obs_list,period,T0,model,energy_range = [0.1,60]*u.TeV)
table = a.phase_to_obs(table)
obs_ids = a.phasewise_obs_list(table)

flux = np.zeros(10)
flux_err = np.zeros(10)

for i in range(10):
    if len(obs_ids[i]) > 1:
        obs_list = data_store.obs_list(obs_ids[i])

        lc = a.evaluate_flux(obs_list,on_region,exclusion_mask)
        flux[i] = lc['FLUX'][0].value
        flux_err[i] = lc['FLUX_ERR'][0].value

In [ ]:
print(flux,flux_err)